<a href="https://colab.research.google.com/github/rjroxx/NLP_Coursework/blob/master/Classwork6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import string 
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split as tts,GridSearchCV
from sklearn.pipeline import Pipeline
!pip install lightgbm
import lightgbm as lgb

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [0]:
sw = set(stopwords.words('english'))

In [0]:
url=("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz")
df = pd.read_json(url,lines=True)
dt = dt.head(1000)
url1="https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv"
dt=pd.read_csv(url1)

In [9]:
Sentence = dt['TweetText'][5]
Sentence

"I'm a current @Blackberry user, little bit disappointed with it! Should I move to @Android or @Apple @iphone"

In [0]:
def c1(Sentence):
  wt = word_tokenize(c2(Sentence.lower()))   
  s = [] 
  for w in wt: 
      if w not in sw: 
          s.append(w) 
  return s  

In [0]:
def c2(data):
  d = data
  translator = str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890')
  d = d.translate(translator)
  return d

In [15]:
sent = c1(Sentence)
sent

['im',
 'current',
 'blackberry',
 'user',
 'little',
 'bit',
 'disappointed',
 'move',
 'android',
 'apple',
 'iphone']

In [0]:
def convert(lst):
  lst = c1(lst)        
  return ' '.join(lst)

In [0]:
def c3(Sentence):# To remove the non english words

  words = set(nltk.corpus.words.words())

  sent = Sentence
  Clean = " ".join(w for w in nltk.wordpunct_tokenize(sent) \
            if w.lower() in words or not w.isalpha())
  return Clean

In [0]:
dt['Cleaned_tweet'] = dt['TweetText'].apply(convert)
dt['Cleaned_tweet'] = dt['Cleaned_tweet'].apply(c2)
dt['Cleaned_tweet'] = dt['Cleaned_tweet'].apply(c3)

In [19]:
dt['Cleaned_tweet']

0                                         apple get crack
1                                   apple carrier support
2       hilarious video guy duet apple pretty much lov...
3                       rim made easy switch apple see ya
4                         reason got twitter thanks apple
                              ...                        
5108                                          con twitter
5109                                              twitter
5110                                twitter la mala de yo
5111                                              twitter
5112                               os para e e twitter da
Name: Cleaned_tweet, Length: 5113, dtype: object

In [20]:
steps = [('text',CountVectorizer()),('ensemble',RandomForestClassifier())]
p = Pipeline(steps)
dt.columns

Index(['Topic', 'Sentiment', 'TweetId', 'TweetDate', 'TweetText',
       'Cleaned_tweet'],
      dtype='object')

In [32]:
vt = CountVectorizer()
X = vt.fit_transform(dt['Cleaned_tweet'])
print(vt.get_feature_names())
x_train,x_test,y_train,y_test = tts(X,dt['Sentiment'],test_size=0.3, random_state=30)

['aa', 'abandoned', 'abbas', 'ability', 'able', 'absolutely', 'abstract', 'academy', 'accelerate', 'accelerated', 'acceleration', 'accent', 'acceptable', 'access', 'accessibility', 'accidently', 'accord', 'according', 'account', 'accuracy', 'accused', 'ach', 'acme', 'acquire', 'acquisition', 'across', 'acting', 'action', 'active', 'activity', 'actual', 'actually', 'ad', 'adat', 'aday', 'add', 'added', 'addict', 'addicted', 'additional', 'address', 'administrator', 'admirably', 'admit', 'adobe', 'adopted', 'adoption', 'adult', 'advance', 'advanced', 'advantage', 'advertise', 'advertisement', 'advertising', 'advisor', 'ae', 'affair', 'affect', 'afford', 'after', 'afternoon', 'afterthought', 'age', 'agency', 'agenda', 'aggressive', 'ago', 'agora', 'agree', 'agricole', 'ah', 'aha', 'ahead', 'ai', 'ailing', 'aim', 'aint', 'air', 'airdrop', 'airport', 'ak', 'aka', 'al', 'ala', 'alamo', 'alarm', 'albatross', 'album', 'alchemy', 'alcohol', 'alert', 'algorithm', 'alive', 'allow', 'ally', 'almos

In [0]:
x_train = pd.DataFrame(x_train.toarray(),columns=vt.get_feature_names())
x_test = pd.DataFrame(x_test.toarray(),columns=vt.get_feature_names())
rf = RandomForestClassifier()
params = {'max_depth': [10, 20, 30, None],'max_features': ['sqrt'],
      'min_samples_leaf': [1, 2, 4],'min_samples_split': [2, 5, 10],
      'n_estimators': range(10,50,10)}
 
gs_rf = GridSearchCV(estimator = rf, param_grid = params, cv = 5, n_jobs = 10, verbose = 1)

In [35]:
gs_rf.fit(x_train,y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   16.4s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:  3.9min
[Parallel(n_jobs=10)]: Done 720 out of 720 | elapsed:  9.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [36]:
print(classification_report(gs_rf.predict(x_test),y_test))

              precision    recall  f1-score   support

  irrelevant       0.93      0.67      0.78       682
    negative       0.37      0.64      0.47        97
     neutral       0.71      0.74      0.73       696
    positive       0.27      0.68      0.39        59

    accuracy                           0.70      1534
   macro avg       0.57      0.68      0.59      1534
weighted avg       0.77      0.70      0.72      1534



In [0]:
params = {
    "objective" : "multiclass",
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':4,
    'learning_rate':0.01,
    'max_depth': 7,
    'num_leaves': 127,
    'feature_fraction': 0.4,
    'bagging_freq': 10,
    'num_iterations':1000 ,
    'max_bin' : 32}


In [0]:
le = LabelEncoder()  
d_train = lgb.Dataset(x_train,le.fit_transform(y_train))

In [42]:
clf = lgb.train( params,d_train, 100)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [46]:
preds = clf.predict(x_test)
best_preds= [np.argmax(line) for line in preds]
np.bincount((le.fit_transform(y_train)))

array([1198,  403, 1607,  371])

In [47]:
print(classification_report(le.inverse_transform(best_preds),y_test))

              precision    recall  f1-score   support

  irrelevant       0.52      0.95      0.67       265
    negative       0.17      0.52      0.26        56
     neutral       0.94      0.58      0.72      1173
    positive       0.13      0.47      0.20        40

    accuracy                           0.64      1534
   macro avg       0.44      0.63      0.46      1534
weighted avg       0.82      0.64      0.68      1534



In [48]:
np.bincount((best_preds))

array([ 265,   56, 1173,   40])

In [49]:
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print(classification_report(le.inverse_transform(best_preds_train),y_train))

              precision    recall  f1-score   support

  irrelevant       0.51      0.98      0.67       628
    negative       0.28      0.67      0.40       168
     neutral       0.96      0.57      0.72      2696
    positive       0.18      0.76      0.29        87

    accuracy                           0.65      3579
   macro avg       0.48      0.75      0.52      3579
weighted avg       0.83      0.65      0.68      3579



In [50]:
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_1(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.6:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.6 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_1(preds)),y_train))
      
pred_rf = gs_rf.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(gs_rf.predict(x_test),y_test)))
print("\n 0.6 threshold for rf : \n",classification_report(le.inverse_transform(thresh_1(pred_rf)),y_test) )

Default Threshold for lgbm : 
               precision    recall  f1-score   support

  irrelevant       0.51      0.98      0.67       628
    negative       0.28      0.67      0.40       168
     neutral       0.96      0.57      0.72      2696
    positive       0.18      0.76      0.29        87

    accuracy                           0.65      3579
   macro avg       0.48      0.75      0.52      3579
weighted avg       0.83      0.65      0.68      3579


 0.6 Threshold for lgbm : 
               precision    recall  f1-score   support

  irrelevant       0.48      0.99      0.65       583
    negative       0.07      0.93      0.13        30
     neutral       0.99      0.54      0.70      2948
    positive       0.05      0.94      0.09        18

    accuracy                           0.62      3579
   macro avg       0.40      0.85      0.39      3579
weighted avg       0.90      0.62      0.68      3579

Default Threshold for rf : 
               precision    recall  f1-sco

In [51]:
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_2(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.7:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.7 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_2(preds)),y_train))

pred_rf = gs_rf.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(gs_rf.predict(x_test),y_test)))
print("\n 0.6 threshold for rf : \n",classification_report(le.inverse_transform(thresh_2(pred_rf)),y_test) )

Default Threshold for lgbm : 
               precision    recall  f1-score   support

  irrelevant       0.51      0.98      0.67       628
    negative       0.28      0.67      0.40       168
     neutral       0.96      0.57      0.72      2696
    positive       0.18      0.76      0.29        87

    accuracy                           0.65      3579
   macro avg       0.48      0.75      0.52      3579
weighted avg       0.83      0.65      0.68      3579


 0.7 Threshold for lgbm : 
               precision    recall  f1-score   support

  irrelevant       0.47      0.99      0.64       573
    negative       0.03      0.93      0.06        14
     neutral       1.00      0.54      0.70      2985
    positive       0.02      1.00      0.04         7

    accuracy                           0.61      3579
   macro avg       0.38      0.86      0.36      3579
weighted avg       0.91      0.61      0.68      3579

Default Threshold for rf : 
               precision    recall  f1-sco

In [52]:
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_3(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.8:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.8 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_3(preds)),y_train))
      
pred_rf = gs_rf.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(gs_rf.predict(x_test),y_test)))
print("\n 0.6 threshold for rf : \n",classification_report(le.inverse_transform(thresh_3(pred_rf)),y_test) )

Default Threshold for lgbm : 
               precision    recall  f1-score   support

  irrelevant       0.51      0.98      0.67       628
    negative       0.28      0.67      0.40       168
     neutral       0.96      0.57      0.72      2696
    positive       0.18      0.76      0.29        87

    accuracy                           0.65      3579
   macro avg       0.48      0.75      0.52      3579
weighted avg       0.83      0.65      0.68      3579


 0.8 Threshold for lgbm : 
               precision    recall  f1-score   support

  irrelevant       0.44      0.99      0.61       534
    negative       0.01      1.00      0.02         5
     neutral       1.00      0.53      0.69      3038
    positive       0.01      1.00      0.01         2

    accuracy                           0.60      3579
   macro avg       0.36      0.88      0.33      3579
weighted avg       0.91      0.60      0.68      3579

Default Threshold for rf : 
               precision    recall  f1-sco